In [2]:
#Trevor O'Hearn
#8/4/20
#Written to Quickly Scrape the overview table of the Federal Reserve

import pandas as pd
import requests
from bs4 import BeautifulSoup
import WebScrapingMethods as wsm

In [3]:
base_url = 'https://www.federalreserve.gov/releases/h41/'

#get list for links
soup = wsm.getSoup(base_url)
hrefs = soup.select('.col-xs-1 a')
end_urls = []
for a in hrefs:
    #get links for 2020 and 2019
    if (a.attrs['href'][:4] == '2020' or a.attrs['href'][:4] == '2019'
        or a.attrs['href'][:4] == 'curr'):
        end_urls.append(a.attrs['href'] + '/h41.htm')
    else:
        break

urls = wsm.getLinks(base_url, end_url_list=end_urls)

In [10]:
url

'https://www.federalreserve.gov/releases/h41/20200702/h41.htm'

In [4]:
url = urls[0]
bs = wsm.getSoup(url)
date = bs.select('.H41Release td p')[0].text.strip()
tables = bs.select('table')

In [30]:
table = tables[1]

In [61]:
tds = table.select('td')

In [62]:
tds

[<td headers="t2c1 t2c3 t2r1" style="font-family:Courier New;font-size:8pt;font-weight:bold;vertical-align:bottom;border-top:solid 1px black;border-right:solid 1px black;padding-right:1.3959em;text-align:right;"><p> 6,975,799</p></td>,
 <td headers="t2c1 t2c4 t2c5 t2r1" style="font-family:Courier New;font-size:8pt;font-weight:bold;vertical-align:bottom;border-top:solid 1px black;border-right:solid 1px black;padding-right:1.3959em;text-align:right;"><p>-   33,865</p></td>,
 <td headers="t2c1 t2c4 t2c6 t2r1" style="font-family:Courier New;font-size:8pt;font-weight:bold;vertical-align:bottom;border-top:solid 1px black;border-right:solid 1px black;padding-right:1.3959em;text-align:right;"><p>+3,194,104</p></td>,
 <td headers="t2c2 t2r1" style="font-family:Courier New;font-size:8pt;font-weight:bold;vertical-align:bottom;border-top:solid 1px black;padding-right:1.3959em;text-align:right;"><p> 6,969,543</p></td>,
 <td headers="t2c1 t2c3 t2r1 t2r2" style="font-family:Courier New;font-size:8pt;

In [63]:
across, down

(1, 2)

In [ ]:
#5 x 36

In [67]:
len(tables)

20

In [71]:
table_columns = []
table_rows = []
table_data = {}
features = wsm.cleanFeatures(wsm.getFeatures(bs))
k = 0
failures = []
for table in tables:
    #get dimensions of table
    try:
        k += 1
        print('in try')
        across = len(table.select('tr')[-1].select('p')) - 1 #can't count the header column
        down = len(table.select('td')) // across
        table_columns.append(across)
        table_rows.append(down)
        
        #get features
        table_num = int(table.select('tr th', limit=1)[0].attrs['id'].strip('t').split('c')[0])
        features[table_num]
        print('got features')
        #get data
        data = []
        i = 0
        while i < down:
            text = tds[across * i + across - 1].text.strip()
            if (len(text) > 0):
                data.append(text)
            i += 1
        print('got data')
        table_data[table_num] = dict(zip(features[table_num], data))
        print('zipped')
        
    except:
        failures.append(k)
        continue
print(failures)
#supposed to fail : 0,3,5, 7, 9, 11, 14, 17, 19 (0 indexed)
#15, 16 need work

in try
in try
got features
got data
zipped
in try
got features
got data
zipped
in try
in try
got features
got data
zipped
in try
in try
got features
in try
in try
got features
got data
zipped
in try
in try
got features
got data
zipped
in try
in try
got features
got data
zipped
in try
got features
got data
zipped
in try
in try
got features
in try
got features
in try
in try
got features
got data
zipped
in try
[1, 4, 6, 7, 8, 10, 12, 15, 16, 17, 18, 20]


In [70]:
table_data

{2: {'Reserve Bank credit': '6,969,543',
  'Securities held outright1': '6,126,948',
  'U.S. Treasury securities': '4,213,220',
  'Bills2': '326,044',
  'Notes and bonds, nominal2': '3,581,139',
  'Notes and bonds, inflation-indexed2': '270,206',
  'Inflation compensation3': '35,831',
  'Federal agency debt securities2': '2,347',
  'Mortgage-backed securities4': '1,911,381',
  'Unamortized premiums on securities held outright5': '314,324',
  'Unamortized discounts on securities held outright5': '-5,345',
  'Repurchase agreements6': '61,201',
  'Foreign official': '1,001',
  'Others': '60,200',
  'Loans': '97,133',
  'Primary credit': '5,860',
  'Secondary credit': '0',
  'Seasonal credit': '16',
  'Primary Dealer Credit Facility': '2,486',
  'Money Market Mutual Fund Liquidity Facility': '20,637',
  'Paycheck Protection Program Liquidity Facility': '68,133',
  'Other credit extensions': '0',
  'Net portfolio holdings of Commercial Paper Funding Facility II LLC7': '12,799',
  'Net portf

In [17]:
len(tables[1].select('tr td')) // across

SelectorSyntaxError: Invalid character '<' position 3
  line 1:
tr < td
   ^ (<string>)

In [12]:

dfs = []
for url in urls:
    bs = wsm.getSoup(url)
    date = bs.select('.H41Release td p')[0].text.strip()
    print(date)
    features = wsm.getFeatures(bs)
    table = bs.select('table')[1]
    tds = table.select('td')
    data = []
    i = 0
    while i < (len(tds) / 4):
        text = tds[4 * i + 3].text.strip()
        if (len(text) > 0):
            data.append(text)
        i += 1
    cleaned_features = wsm.cleanFeatures(features)

    dfvals = {}
    dfvals['Date'] = date
    for i, f in enumerate(cleaned_features[2]):
        dfvals[f] = data[i]

    df = pd.DataFrame(data=dfvals, index=[0])
    dfs.append(df)

#clean dataframe
df = dfs[0]
for d in dfs[1:]:
    df = df.append(d, ignore_index=False)
df['Date'] = pd.to_datetime(df['Date'], format='%B %d, %Y', errors='ignore')
df.set_index('Date', inplace=True)

all_dfs = [df]
columnNames = df.columns
df.fillna('0', inplace=True)
#clean data frame
for df in all_dfs:
    for c in columnNames:
        df[c] = df[c].apply(wsm.removeUnicode)
        df[c] = df[c].apply(wsm.removePlus)
        df[c] = df[c].apply(wsm.removeComma)
        df[c] = df[c].astype(int)
#df_all.to_csv('quickscrape.csv')


July 2, 2020
July 9, 2020
July 16, 2020
July 23, 2020
July 30, 2020
June 4, 2020
June 11, 2020
June 18, 2020
June 25, 2020
May 7, 2020
May 14, 2020
May 21, 2020
May 28, 2020
April 2, 2020
April 9, 2020
April 16, 2020
April 23, 2020
April 30, 2020
March 5, 2020
March 12, 2020
March 19, 2020
March 26, 2020
February 6, 2020
February 13, 2020
February 20, 2020
February 27, 2020
January 2, 2020
January 9, 2020
January 16, 2020
January 23, 2020
January 30, 2020
December 5, 2019
December 12, 2019
December 19, 2019
December 26, 2019
November 7, 2019
November 14, 2019
November 21, 2019
November 29, 2019
October 3, 2019
October 10, 2019
October 17, 2019
October 24, 2019
October 31, 2019
September 5, 2019
September 12, 2019
September 19, 2019
September 26, 2019
August 1, 2019
August 8, 2019
August 15, 2019
August 22, 2019
August 29, 2019
July 5, 2019
July 11, 2019
July 18, 2019
July 25, 2019
June 6, 2019
June 13, 2019
June 20, 2019
June 27, 2019
May 2, 2019
May 9, 2019
May 16, 2019
May 23, 2019
M

In [2]:
dfvals

{'Date': 'July 2, 2020',
 'Reserve Bank credit': '6,969,543',
 'Securities held outright1': '6,126,948',
 'U.S. Treasury securities': '4,213,220',
 'Bills2': '326,044',
 'Notes and bonds, nominal2': '3,581,139',
 'Notes and bonds, inflation-indexed2': '270,206',
 'Inflation compensation3': '35,831',
 'Federal agency debt securities2': '2,347',
 'Mortgage-backed securities4': '1,911,381',
 'Unamortized premiums on securities held outright5': '314,324',
 'Unamortized discounts on securities held outright5': '-5,345',
 'Repurchase agreements6': '61,201',
 'Foreign official': '1,001',
 'Others': '60,200',
 'Loans': '97,133',
 'Primary credit': '5,860',
 'Secondary credit': '0',
 'Seasonal credit': '16',
 'Primary Dealer Credit Facility': '2,486',
 'Money Market Mutual Fund Liquidity Facility': '20,637',
 'Paycheck Protection Program Liquidity Facility': '68,133',
 'Other credit extensions': '0',
 'Net portfolio holdings of Commercial Paper Funding Facility II LLC7': '12,799',
 'Net portfol

In [4]:
df = pd.DataFrame(data=dfvals, index=[0])

In [7]:
df2 = df.copy()

In [9]:
type(df)

dict

In [10]:
dfs

[{'Date': 'July 2, 2020',
  'Reserve Bank credit': '6,969,543',
  'Securities held outright1': '6,126,948',
  'U.S. Treasury securities': '4,213,220',
  'Bills2': '326,044',
  'Notes and bonds, nominal2': '3,581,139',
  'Notes and bonds, inflation-indexed2': '270,206',
  'Inflation compensation3': '35,831',
  'Federal agency debt securities2': '2,347',
  'Mortgage-backed securities4': '1,911,381',
  'Unamortized premiums on securities held outright5': '314,324',
  'Unamortized discounts on securities held outright5': '-5,345',
  'Repurchase agreements6': '61,201',
  'Foreign official': '1,001',
  'Others': '60,200',
  'Loans': '97,133',
  'Primary credit': '5,860',
  'Secondary credit': '0',
  'Seasonal credit': '16',
  'Primary Dealer Credit Facility': '2,486',
  'Money Market Mutual Fund Liquidity Facility': '20,637',
  'Paycheck Protection Program Liquidity Facility': '68,133',
  'Other credit extensions': '0',
  'Net portfolio holdings of Commercial Paper Funding Facility II LLC7':

In [19]:
columns = all_dfs[0].columns

In [21]:
columns

Index(['Bills2', 'Central bank liquidity swaps8',
       'Federal agency debt securities2', 'Float',
       'Foreign currency denominated assets10', 'Foreign official',
       'Gold stock', 'Inflation compensation3', 'Loans',
       'Money Market Mutual Fund Liquidity Facility',
       'Mortgage-backed securities4',
       'Net portfolio holdings of Commercial Paper Funding Facility II LLC7',
       'Net portfolio holdings of Corporate Credit Facilities LLC7',
       'Net portfolio holdings of Corporate Credit Facility LLC7',
       'Net portfolio holdings of MS Facilities LLC (Main Street Lending Program)7',
       'Net portfolio holdings of Maiden Lane LLC7',
       'Net portfolio holdings of Municipal Liquidity Facility LLC7',
       'Net portfolio holdings of TALF II LLC7',
       'Notes and bonds, inflation-indexed2', 'Notes and bonds, nominal2',
       'Other Federal Reserve assets9', 'Other credit extensions', 'Others',
       'Paycheck Protection Program Liquidity Facility',
  

In [25]:
columns[18]

'Notes and bonds, inflation-indexed2'